In [1]:
# -*- coding: utf-8 -*-

import os
import math
import argparse
import random
import numpy
import torch
import torch.nn as nn
from bucket_iterator import BucketIterator
from sklearn import metrics
from data_utils import ABSADatesetReader
from models import LC_GCN

class Instructor:
    def __init__(self, opt):
        self.opt = opt

        absa_dataset = ABSADatesetReader(dataset=opt.dataset, embed_dim=opt.embed_dim)

        self.train_data_loader = BucketIterator(data=absa_dataset.train_data, batch_size=opt.batch_size, shuffle=True)
        self.test_data_loader = BucketIterator(data=absa_dataset.test_data, batch_size=opt.batch_size, shuffle=False)

        self.model = opt.model_class(absa_dataset.embedding_matrix, opt).to(opt.device)
        self._print_args()
        self.global_f1 = 0.

        if torch.cuda.is_available():
            print('cuda memory allocated:', torch.cuda.memory_allocated(device=opt.device.index))

    def _print_args(self):
        n_trainable_params, n_nontrainable_params = 0, 0
        for p in self.model.parameters():
            n_params = torch.prod(torch.tensor(p.shape)).item()
            if p.requires_grad:
                n_trainable_params += n_params
            else:
                n_nontrainable_params += n_params
        print('n_trainable_params: {0}, n_nontrainable_params: {1}'.format(n_trainable_params, n_nontrainable_params))
        print('> training arguments:')
        for arg in vars(self.opt):
            print('>>> {0}: {1}'.format(arg, getattr(self.opt, arg)))

    def _reset_params(self):
        for p in self.model.parameters():
            if p.requires_grad:
                if len(p.shape) > 1:
                    self.opt.initializer(p)
                else:
                    stdv = 1. / math.sqrt(p.shape[0])
                    torch.nn.init.uniform_(p, a=-stdv, b=stdv)

    def _train(self, criterion, optimizer):
        max_test_acc = 0
        max_test_f1 = 0
        global_step = 0
        continue_not_increase = 0
        for epoch in range(self.opt.num_epoch):
            print('>' * 100)
            print('epoch: ', epoch)
            n_correct, n_total = 0, 0
            increase_flag = False
            for i_batch, sample_batched in enumerate(self.train_data_loader):
                global_step += 1

                # switch model to training mode, clear gradient accumulators
                self.model.train()
                optimizer.zero_grad()

                inputs = [sample_batched[col].to(self.opt.device) for col in self.opt.inputs_cols]
                targets = sample_batched['polarity'].to(self.opt.device)

                outputs = self.model(inputs)
                loss = criterion(outputs, targets)
                loss.backward()
                optimizer.step()

                if global_step % self.opt.log_step == 0:
                    n_correct += (torch.argmax(outputs, -1) == targets).sum().item()
                    n_total += len(outputs)
                    train_acc = n_correct / n_total

                    test_acc, test_f1 = self._evaluate_acc_f1()
                    if test_acc > max_test_acc:
                        max_test_acc = test_acc
                    if test_f1 > max_test_f1:
                        increase_flag = True
                        max_test_f1 = test_f1
                        if self.opt.save and test_f1 > self.global_f1:
                            self.global_f1 = test_f1
                            torch.save(self.model.state_dict(), 'state_dict/'+self.opt.model_name+'_'+self.opt.dataset+'.pkl')
                            print('>>> best model saved.')
                    print('loss: {:.4f}, acc: {:.4f}, test_acc: {:.4f}, test_f1: {:.4f}'.format(loss.item(), train_acc, test_acc, test_f1))
            if increase_flag == False:
                continue_not_increase += 1
                if continue_not_increase >= 5:
                    print('early stop.')
                    break
            else:
                continue_not_increase = 0    
        return max_test_acc, max_test_f1

    def _evaluate_acc_f1(self):
        # switch model to evaluation mode
        self.model.eval()
        n_test_correct, n_test_total = 0, 0
        t_targets_all, t_outputs_all = None, None
        with torch.no_grad():
            for t_batch, t_sample_batched in enumerate(self.test_data_loader):
                t_inputs = [t_sample_batched[col].to(opt.device) for col in self.opt.inputs_cols]
                t_targets = t_sample_batched['polarity'].to(opt.device)
                t_outputs = self.model(t_inputs)

                n_test_correct += (torch.argmax(t_outputs, -1) == t_targets).sum().item()
                n_test_total += len(t_outputs)

                if t_targets_all is None:
                    t_targets_all = t_targets
                    t_outputs_all = t_outputs
                else:
                    t_targets_all = torch.cat((t_targets_all, t_targets), dim=0)
                    t_outputs_all = torch.cat((t_outputs_all, t_outputs), dim=0)

        test_acc = n_test_correct / n_test_total
        f1 = metrics.f1_score(t_targets_all.cpu(), torch.argmax(t_outputs_all, -1).cpu(), labels=[0, 1, 2], average='macro')
        return test_acc, f1

    def run(self, repeats=10):
        # Loss and Optimizer
        criterion = nn.CrossEntropyLoss()
        
        if not os.path.exists('log/'):
            os.mkdir('log/')

        f_out = open('log/'+self.opt.model_name+'_'+self.opt.dataset+str(self.opt.SRD)+'_val.txt', 'w', encoding='utf-8')

        max_test_acc_avg = 0
        max_test_f1_avg = 0
        for i in range(repeats):
            print('repeat: ', (i+1))
            f_out.write('repeat: '+str(i+1))
            self._reset_params()
            _params = filter(lambda p: p.requires_grad, self.model.parameters())
            optimizer = self.opt.optimizer(_params, lr=self.opt.learning_rate, weight_decay=self.opt.l2reg)
            max_test_acc, max_test_f1 = self._train(criterion, optimizer)
            print('test_acc: {0}     test_f1: {1}'.format(max_test_acc, max_test_f1))
            f_out.write('test_acc: {0}, test_f1: {1}'.format(max_test_acc, max_test_f1))
            f_out.write('\n')
            max_test_acc_avg += max_test_acc
            max_test_f1_avg += max_test_f1
            print('#' * 100)
        print("test_acc_avg:", max_test_acc_avg / repeats)
        print("test_f1_avg:", max_test_f1_avg / repeats)
        f_out.write('test_acc: {0}, test_f1: {1}'.format(max_test_acc_avg / repeats, max_test_f1_avg / repeats))

        f_out.close()


if __name__ == '__main__':
    # Hyper Parameters
    parser = argparse.ArgumentParser()
    parser.add_argument('--model_name', default='lcgcn', type=str)
    parser.add_argument('--dataset', default='rest14', type=str, help='rest14, lap14')
    parser.add_argument('--optimizer', default='adam', type=str)
    parser.add_argument('--initializer', default='xavier_uniform_', type=str)
    parser.add_argument('--learning_rate', default=0.003, type=float)
    parser.add_argument('--l2reg', default=0.00001, type=float)
    parser.add_argument('--num_epoch', default=100, type=int)
    parser.add_argument('--batch_size', default=32, type=int)
    parser.add_argument('--log_step', default=5, type=int)
    parser.add_argument('--embed_dim', default=300, type=int)
    parser.add_argument('--hidden_dim', default=300, type=int)
    parser.add_argument('--polarities_dim', default=3, type=int)
    parser.add_argument('--save', default=False, type=bool)
    parser.add_argument('--seed', default=776, type=int)
    parser.add_argument('--device', default=None, type=str)
    parser.add_argument('--SRD', default=8, type=int, help='semantic-relative-distance')
    opt = parser.parse_args([])

    model_classes = {
        'lcgcn': LC_GCN,
    }
    input_colses = {
        'lcgcn': ['text_indices', 'aspect_indices', 'left_indices', 'dependency_graph', 'dependency_tree'],
    }
    initializers = {
        'xavier_uniform_': torch.nn.init.xavier_uniform_,
        'xavier_normal_': torch.nn.init.xavier_normal_,
        'orthogonal_': torch.nn.init.orthogonal_,
    }
    optimizers = {
        'adadelta': torch.optim.Adadelta,  
        'adagrad': torch.optim.Adagrad,  
        'adam': torch.optim.Adam,  
        'adamax': torch.optim.Adamax,  
        'asgd': torch.optim.ASGD,  
        'rmsprop': torch.optim.RMSprop,  
        'sgd': torch.optim.SGD,
    }
    opt.model_class = model_classes[opt.model_name]
    opt.inputs_cols = input_colses[opt.model_name]
    opt.initializer = initializers[opt.initializer]
    opt.optimizer = optimizers[opt.optimizer]
    opt.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') \
        if opt.device is None else torch.device(opt.device)

    if opt.seed is not None:
        random.seed(opt.seed)
        numpy.random.seed(opt.seed)
        torch.manual_seed(opt.seed)
        torch.cuda.manual_seed(opt.seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

    ins = Instructor(opt)
    ins.run()


preparing rest14 dataset ...
loading rest14 tokenizer...
loading embedding_matrix: 300_rest14_embedding_matrix.pkl
n_trainable_params: 903903, n_nontrainable_params: 1375500
> training arguments:
>>> model_name: lcgcn
>>> dataset: rest14
>>> optimizer: <class 'torch.optim.adam.Adam'>
>>> initializer: <function xavier_uniform_ at 0x0000012808622DC8>
>>> learning_rate: 0.003
>>> l2reg: 1e-05
>>> num_epoch: 100
>>> batch_size: 32
>>> log_step: 5
>>> embed_dim: 300
>>> hidden_dim: 300
>>> polarities_dim: 3
>>> save: False
>>> seed: 776
>>> device: cuda
>>> SRD: 8
>>> model_class: <class 'models.lcgcn3.LC_GCN'>
>>> inputs_cols: ['text_indices', 'aspect_indices', 'left_indices', 'dependency_graph', 'dependency_tree']
cuda memory allocated: 9120768
repeat:  1
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
epoch:  0
loss: 0.8521, acc: 0.6875, test_acc: 0.6509, test_f1: 0.2666


C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


loss: 0.7366, acc: 0.7500, test_acc: 0.6661, test_f1: 0.3248
loss: 0.8931, acc: 0.7188, test_acc: 0.6536, test_f1: 0.2766
loss: 0.9173, acc: 0.6875, test_acc: 0.7089, test_f1: 0.4752
loss: 0.6029, acc: 0.7312, test_acc: 0.6813, test_f1: 0.3823
loss: 1.3102, acc: 0.6562, test_acc: 0.6723, test_f1: 0.3593
loss: 0.7622, acc: 0.6607, test_acc: 0.6446, test_f1: 0.4553
loss: 0.6924, acc: 0.6680, test_acc: 0.7295, test_f1: 0.5280
loss: 0.4357, acc: 0.6840, test_acc: 0.6920, test_f1: 0.4216
loss: 0.8795, acc: 0.6719, test_acc: 0.7018, test_f1: 0.4391
loss: 0.4802, acc: 0.6875, test_acc: 0.7223, test_f1: 0.5149
loss: 1.0312, acc: 0.6771, test_acc: 0.7616, test_f1: 0.6377
loss: 0.8790, acc: 0.6707, test_acc: 0.7696, test_f1: 0.6294
loss: 0.5513, acc: 0.6741, test_acc: 0.7321, test_f1: 0.5168
loss: 0.6521, acc: 0.6750, test_acc: 0.7554, test_f1: 0.5638
loss: 0.6172, acc: 0.6777, test_acc: 0.7616, test_f1: 0.5828
loss: 0.4448, acc: 0.6857, test_acc: 0.7295, test_f1: 0.5320
loss: 0.3762, acc: 0.697

loss: 0.5790, acc: 0.8021, test_acc: 0.7804, test_f1: 0.6417
loss: 0.4119, acc: 0.8008, test_acc: 0.7839, test_f1: 0.6654
loss: 0.5522, acc: 0.7978, test_acc: 0.7920, test_f1: 0.6855
loss: 0.4661, acc: 0.7986, test_acc: 0.7964, test_f1: 0.6884
loss: 0.5373, acc: 0.7993, test_acc: 0.7982, test_f1: 0.6820
loss: 0.2987, acc: 0.8031, test_acc: 0.7991, test_f1: 0.6762
loss: 0.4862, acc: 0.8051, test_acc: 0.8036, test_f1: 0.6885
loss: 0.2978, acc: 0.8097, test_acc: 0.7991, test_f1: 0.6959
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
epoch:  6
loss: 0.7117, acc: 0.5938, test_acc: 0.8045, test_f1: 0.7124
loss: 0.4086, acc: 0.7188, test_acc: 0.8107, test_f1: 0.7046
loss: 0.3867, acc: 0.7500, test_acc: 0.8054, test_f1: 0.6923
loss: 0.4408, acc: 0.7500, test_acc: 0.8089, test_f1: 0.7069
loss: 0.5195, acc: 0.7625, test_acc: 0.8071, test_f1: 0.7229
loss: 0.3528, acc: 0.7760, test_acc: 0.8018, test_f1: 0.7067
loss: 0.2164, acc: 0.7902, test_acc

loss: 0.4156, acc: 0.8125, test_acc: 0.7607, test_f1: 0.6272
loss: 0.6779, acc: 0.7812, test_acc: 0.7679, test_f1: 0.6295
loss: 0.7143, acc: 0.7396, test_acc: 0.7768, test_f1: 0.6201
loss: 0.6994, acc: 0.7344, test_acc: 0.7741, test_f1: 0.6388
loss: 0.4755, acc: 0.7562, test_acc: 0.7759, test_f1: 0.6631
loss: 1.0090, acc: 0.7396, test_acc: 0.7705, test_f1: 0.6454
loss: 0.6705, acc: 0.7455, test_acc: 0.7777, test_f1: 0.6448
loss: 0.9551, acc: 0.7305, test_acc: 0.7848, test_f1: 0.6645
loss: 0.8411, acc: 0.7222, test_acc: 0.7893, test_f1: 0.6794
loss: 0.7104, acc: 0.7156, test_acc: 0.7973, test_f1: 0.6876
loss: 0.3512, acc: 0.7301, test_acc: 0.7741, test_f1: 0.6248
loss: 0.4961, acc: 0.7292, test_acc: 0.7920, test_f1: 0.6757
loss: 0.7004, acc: 0.7308, test_acc: 0.7884, test_f1: 0.6682
loss: 0.2376, acc: 0.7478, test_acc: 0.7875, test_f1: 0.6659
loss: 0.7878, acc: 0.7375, test_acc: 0.7911, test_f1: 0.6741
loss: 0.8539, acc: 0.7363, test_acc: 0.7714, test_f1: 0.6812
loss: 0.7307, acc: 0.733

...

loss: 0.5945, acc: 0.7885, test_acc: 0.7929, test_f1: 0.6694
loss: 0.2010, acc: 0.7969, test_acc: 0.7929, test_f1: 0.6600
loss: 0.4152, acc: 0.8000, test_acc: 0.7884, test_f1: 0.6663
loss: 0.5174, acc: 0.7949, test_acc: 0.7955, test_f1: 0.6789
loss: 0.5558, acc: 0.7960, test_acc: 0.7964, test_f1: 0.6780
loss: 0.3568, acc: 0.8003, test_acc: 0.7929, test_f1: 0.6687
loss: 0.4675, acc: 0.8026, test_acc: 0.7848, test_f1: 0.6359
loss: 0.7631, acc: 0.7969, test_acc: 0.7911, test_f1: 0.6519
loss: 0.5713, acc: 0.7991, test_acc: 0.7991, test_f1: 0.6906
loss: 0.4521, acc: 0.7983, test_acc: 0.7973, test_f1: 0.6991
loss: 0.5892, acc: 0.7976, test_acc: 0.7964, test_f1: 0.6714
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
epoch:  5
loss: 0.2742, acc: 0.9375, test_acc: 0.7911, test_f1: 0.6563
loss: 0.2287, acc: 0.9219, test_acc: 0.7875, test_f1: 0.6561
loss: 0.3859, acc: 0.8958, test_acc: 0.8018, test_f1: 0.7080
loss: 0.1345, acc: 0.9219, test_acc

loss: 0.2486, acc: 0.9062, test_acc: 0.8036, test_f1: 0.7045
loss: 0.5725, acc: 0.8438, test_acc: 0.8071, test_f1: 0.7097
loss: 0.0861, acc: 0.8958, test_acc: 0.8054, test_f1: 0.7057
loss: 0.0638, acc: 0.9219, test_acc: 0.8080, test_f1: 0.7097
loss: 0.4175, acc: 0.9062, test_acc: 0.7991, test_f1: 0.7072
loss: 0.2245, acc: 0.9062, test_acc: 0.7911, test_f1: 0.6872
loss: 0.1282, acc: 0.9196, test_acc: 0.7929, test_f1: 0.6807
loss: 0.2489, acc: 0.9219, test_acc: 0.7857, test_f1: 0.6606
loss: 0.1905, acc: 0.9201, test_acc: 0.7804, test_f1: 0.6644
loss: 0.1614, acc: 0.9250, test_acc: 0.7937, test_f1: 0.6988
loss: 0.3333, acc: 0.9233, test_acc: 0.7920, test_f1: 0.6930
loss: 0.1440, acc: 0.9245, test_acc: 0.7893, test_f1: 0.6728
loss: 0.1889, acc: 0.9255, test_acc: 0.8018, test_f1: 0.7020
loss: 0.3696, acc: 0.9196, test_acc: 0.7911, test_f1: 0.6905
loss: 0.6199, acc: 0.9083, test_acc: 0.7937, test_f1: 0.6900
loss: 0.1800, acc: 0.9121, test_acc: 0.7911, test_f1: 0.6819
loss: 0.1276, acc: 0.915

loss: 0.6149, acc: 0.7396, test_acc: 0.7812, test_f1: 0.6796
loss: 0.5673, acc: 0.7404, test_acc: 0.7705, test_f1: 0.6467
loss: 0.7822, acc: 0.7411, test_acc: 0.7679, test_f1: 0.6150
loss: 0.6445, acc: 0.7438, test_acc: 0.7804, test_f1: 0.6478
loss: 0.7519, acc: 0.7402, test_acc: 0.7857, test_f1: 0.6493
loss: 0.7470, acc: 0.7369, test_acc: 0.7750, test_f1: 0.6316
loss: 0.7913, acc: 0.7359, test_acc: 0.7821, test_f1: 0.6559
loss: 0.5073, acc: 0.7433, test_acc: 0.7830, test_f1: 0.6536
loss: 0.7383, acc: 0.7405, test_acc: 0.7839, test_f1: 0.6656
loss: 0.2290, acc: 0.7515, test_acc: 0.7920, test_f1: 0.6785
loss: 0.4632, acc: 0.7514, test_acc: 0.7911, test_f1: 0.6737
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
epoch:  3
loss: 0.6349, acc: 0.6562, test_acc: 0.7893, test_f1: 0.6542
loss: 0.5408, acc: 0.7344, test_acc: 0.8018, test_f1: 0.6911
loss: 0.7745, acc: 0.7292, test_acc: 0.7973, test_f1: 0.6820
loss: 0.5387, acc: 0.7500, test_acc

loss: 0.2174, acc: 0.9062, test_acc: 0.8036, test_f1: 0.7141
loss: 0.2735, acc: 0.8906, test_acc: 0.8063, test_f1: 0.7167
loss: 0.1624, acc: 0.9062, test_acc: 0.7937, test_f1: 0.6835
loss: 0.2630, acc: 0.9062, test_acc: 0.7911, test_f1: 0.6713
loss: 0.2252, acc: 0.9125, test_acc: 0.7920, test_f1: 0.6872
loss: 0.4098, acc: 0.9010, test_acc: 0.7884, test_f1: 0.6928
loss: 0.5173, acc: 0.8884, test_acc: 0.7875, test_f1: 0.6731
loss: 0.3839, acc: 0.8789, test_acc: 0.7946, test_f1: 0.6802
loss: 0.2314, acc: 0.8819, test_acc: 0.8018, test_f1: 0.7008
loss: 0.3457, acc: 0.8812, test_acc: 0.7884, test_f1: 0.6640
loss: 0.2292, acc: 0.8864, test_acc: 0.7991, test_f1: 0.6954
loss: 0.0687, acc: 0.8958, test_acc: 0.7982, test_f1: 0.7001
loss: 0.5535, acc: 0.8798, test_acc: 0.8098, test_f1: 0.7164
loss: 0.4387, acc: 0.8817, test_acc: 0.8036, test_f1: 0.7099
loss: 0.2617, acc: 0.8854, test_acc: 0.8036, test_f1: 0.7131
loss: 0.5356, acc: 0.8828, test_acc: 0.7955, test_f1: 0.6915
loss: 0.2784, acc: 0.886

loss: 0.2000, acc: 0.9464, test_acc: 0.7884, test_f1: 0.6995
loss: 0.2381, acc: 0.9417, test_acc: 0.7946, test_f1: 0.7003
loss: 0.4238, acc: 0.9336, test_acc: 0.7946, test_f1: 0.6979
loss: 0.0906, acc: 0.9357, test_acc: 0.7982, test_f1: 0.7095
loss: 0.1081, acc: 0.9392, test_acc: 0.8089, test_f1: 0.7249
loss: 0.2320, acc: 0.9342, test_acc: 0.8009, test_f1: 0.7056
loss: 0.1305, acc: 0.9359, test_acc: 0.7946, test_f1: 0.6928
loss: 0.0308, acc: 0.9390, test_acc: 0.7982, test_f1: 0.6995
loss: 0.2883, acc: 0.9361, test_acc: 0.7929, test_f1: 0.6962
loss: 0.1462, acc: 0.9375, test_acc: 0.7955, test_f1: 0.7026
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
epoch:  14
loss: 0.0181, acc: 1.0000, test_acc: 0.7955, test_f1: 0.6954
loss: 0.0273, acc: 1.0000, test_acc: 0.7964, test_f1: 0.6951
loss: 0.0666, acc: 0.9896, test_acc: 0.7973, test_f1: 0.6988
loss: 0.5213, acc: 0.9609, test_acc: 0.7866, test_f1: 0.6833
loss: 0.1162, acc: 0.9625, test_ac